In [1]:
%cd /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt

/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt


In [2]:
path = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/'

CATEGORIES = ['MGMT_negative', 'MGMT_positive']

block_h, block_w = (32, 32)  # Image size for Blocks of images
stride = 3  # Enter the value of stride to increment the loop

Case_Num = 5  # Creating Number of classes

In [3]:
# Function Definition --> Backup Original Data For Safety Purpose

def Backup():
    import shutil
    print("Creating Backup")
    # Copy data for backup
    try:
        shutil.copytree(PATH, Org_Dir)  # Syntax --> shutil.copytree(Source, Destination)
    except Exception as e:
        pass

    # Copy data for working
    try:
        shutil.copytree(Org_Dir, Work_Dir)
    except Exception as e:
        pass

    # Deleting folder, listed in the CATEGORIES list, after creating Cases
    for cate in CATEGORIES:
        try:
            shutil.rmtree(PATH + cate)  # Deleting Folders of CATEGORIES list
        except Exception as e:
            pass

In [4]:
# Function Definition --> Generate Tumor Images of Patients
# Process
# 1. Loading data to nii file
# 2. Converting nii data to 3d np array
# 3. Extracting layers before multiplying, analysing segmentation data, if mask value is zero for that layer we won't be extracting it
# 4. Getting range of layer numbers, from where segmentation data is relevant
# 5. Creating new mask to be later multiplied with segmentation data
# 6. Again checking for zero value layers and removing them
# 7. Converting the np array back to png Generate_images
# 8. Resizing the mask data then resizing them (32,32)
# 9. Deleting the nii files to save storage

def Generate_images():
    import os
    print('Generating Images')
    workdir= os.listdir(Work_Dir)
    if '.DS_Store' in workdir:
        workdir.remove('.DS_Store')
        print('Removed .DS_Store from directory list')
        print('Work Directory: ', workdir)
    for Type in workdir:
        Type_Path = os.path.join(Work_Dir, Type + '/')
        print('Type_Path: ', Type_Path)
        try:
            typedir = os.listdir(Type_Path)
            if '.DS_Store' in typedir:
                typedir.remove('.DS_Store')
                print('Removed .DS_Store')
                print('Type_Path Directory: ', typedir)
                
            for patient in typedir:
                print('Resolving patient number {}'.format(patient))
                Patient_Path = os.path.join(Type_Path, patient + '/')
                os.chdir(Patient_Path)
                flair = nib.load('{}_flair.nii.gz'.format(patient))
                flair_data = flair.get_fdata() #Converting nii to 3d np array
                seg_mask = nib.load('{}_seg.nii.gz'.format(patient))
                seg_mask_data = seg_mask.get_fdata() #Converting nii to 3d np array

                #Extracting only those layers from mask hich have non zero values
                z = np.any(seg_mask_data, axis=(0, 1))
                zmin, zmax = np.where(z)[0][[0, -1]] #zmin & zmax saves the corresponding layer numbers of tumor regions

                #Creating a new mask to remove segmentation
                d = seg_mask_data
                for layer in range(zmin,zmax+1):
                    nonzero = np.nonzero(d[:,:,layer])
                    r = nonzero[0]
                    c = nonzero[1]
                    if (r.size == 0 or c.size == 0):
                        continue
                    rmin = np.min(r)
                    rmax = np.max(r)
                    cmin = np.min(c)
                    cmax = np.max(c)
                    d[rmin:rmax+1, cmin:cmax+1, layer] = 1 #Replacing tumor region values by 1

                #Multiplying flair data with new mask
                tumor = np.multiply(flair_data,d)

                #Removing zero valued layers
                tumor_layers = tumor[:,:,~(tumor==0).all((0,1))]

                #converting to png files
                Cropped_list = [] #list containing cropped 2d layers of tumor region
                for lay in range(0,tumor_layers.shape[2]):
                    coords = np.argwhere(tumor_layers[:,:,lay])
                    x_min, y_min = coords.min(axis=0)
                    x_max, y_max = coords.max(axis=0)
                    cropped = tumor_layers[x_min:x_max+1, y_min:y_max+1, lay]
                    cropped *= (255.0/cropped.max()) #normalization/scaling
                    Cropped_list.append(cropped)

                # Checking for height and width and resizing them
                frame = 0
                for item in Cropped_list:
                    if((item.shape[0]*item.shape[1]) >= 300):
                        frame = frame + 1
                        im = Image.fromarray(item)
                        im = im.convert('L')
                        width, height = im.size
                        if(height < 32 and width >= 32):
                            im = im.resize((32, width))
                        elif(height >= 32 and width < 32):
                            im = im.resize((height, 32))
                        elif(height < 32 and width < 32):
                            im = im.resize((32,32))
                        im.save("{}_img_{}.jpeg".format(patient,frame))
                        im.close()

                #Removing unwanted nii files
                osList=os.listdir(os.getcwd())
                if '.DS_Store' in osList:
                    osList.remove('.DS_Store')
                    print('Removed .DS_Store from osList')
                for item in osList:
                    try:
                        if item.endswith(".gz"):
                            os.remove(item)
                    except Exception as e:
                        pass
        except Exception as e:
            print('Error Found',e)
    os.chdir(Work_Dir)

In [5]:
# Function Definition --> Slice Images into Blocks
# Here we are slicing the Image in blocks

def Creating_Image_Blocks(block_h, block_w, stride):
    import glob
    from PIL import Image
    from tqdm import tqdm
    import shutil

    osdir = os.listdir(Work_Dir)
    if '.DS_Store' in osdir:
        osdir.remove('.DS_Store')
        print('Removed .DS_Store')
        print('OS DIRECTORY: ', osdir)
    for category in tqdm(osdir):
        Category_Path = os.path.join(Work_Dir, category + '/')
        categorydir = os.listdir(Category_Path)
        if '.DS_Store' in categorydir:
            categorydir.remove('.DS_Store')
        for patient in categorydir:
            Patient_Path = os.path.join(Category_Path, patient + '/')

            os.chdir(Patient_Path)
            for img_file in glob.glob('*.jpeg'):
                img = Image.open(img_file)
                img_w, img_h = img.size

                File_Name, extension = os.path.splitext(img_file)

                SaveDir = Patient_Path

                frame_num = 0
                count_row = 0

                for row in range(0, img_h, stride):
                    if (img_h - row >= block_h):
                        count_row += 1
                        count_column = 0

                        for col in range(0, img_w, stride):
                            if (img_w - col >= block_w):
                                count_column += 1
                                # Syntax --> img.crop((left, top, right, bottom))
                                crop = img.crop(
                                    (col, row, col + block_w, row + block_h))
                                save_to = os.path.join(
                                    SaveDir + '/' + f'{File_Name}_{count_row}_{count_column}.jpg')
                                crop.save(save_to.format(frame_num + 1))

                img.close()
                # Deleting Image Files
                os.remove(img_file)

    os.chdir(path)


In [8]:
# All Function Calls
import os
import numpy as np
import nibabel as nib
from PIL import Image

PATH = path + 'Data/'
Org_Dir = PATH + 'Original_Data_Backup/'
Work_Dir = PATH + 'Working_Data/'

# Function Call --> Backup Original Data For Safety Purpose
Backup()

# Function Call --> Generate Tumor Images of Patients

Generate_images()

# Function Call --> Slice Images into Blocks
Creating_Image_Blocks(block_h, block_w, stride)

os.chdir(path)

Generating Images
Removed .DS_Store from directory list
Work Directory:  ['MGMT_positive', 'MGMT_negative']
Type_Path:  /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/Working_Data/MGMT_positive/
Removed .DS_Store
Type_Path Directory:  ['BraTS2021_00005', 'BraTS2021_00002', 'BraTS2021_00000', 'BraTS2021_00006', 'BraTS2021_00008']
Resolving patient number BraTS2021_00005
Resolving patient number BraTS2021_00002
Resolving patient number BraTS2021_00000
Resolving patient number BraTS2021_00006
Resolving patient number BraTS2021_00008
Type_Path:  /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/Working_Data/MGMT_negative/
Removed .DS_Store
Type_Path Directory:  ['BraTS2021_00003', 'BraTS2021_00018', 'BraTS2021_00019', 'BraTS2021_00017', 'BraTS2021_00009']
Resolving patient number BraTS2021_00003
Resolving patient number BraTS2021_00018
Resolving patient number BraTS2021_00019
Resolving patient number BraTS2021_00017
Resolving patient number Br

100%|██████████| 2/2 [00:00<00:00, 1319.17it/s]
